In [ ]:
from nidaqmx import Task
from nidaqmx.constants import Edge, AcquisitionType
import numpy as np

from sashimi.hardware.cameras.hamamatsu.interface import HamamatsuCamera, TriggerMode
import napari
import pyqtgraph as pg
import time

In [ ]:
max_res = 2048

camera = HamamatsuCamera(0, (max_res, max_res))

camera.exposure_time = 10
camera.trigger_mode = TriggerMode.EXTERNAL_TRIGGER

In [ ]:
task = Task()
task.ao_channels.add_ao_voltage_chan("Dev2/ao3", min_val=-10, max_val=10)
task.timing.cfg_samp_clk_timing(
    rate=100,
    source="OnboardClock",
    active_edge=Edge.RISING,
    sample_mode=AcquisitionType.CONTINUOUS,
    samps_per_chan=1000,
)

In [ ]:
# 5: on, 0: off

# trigger = np.array(([5, 0] * 50))
trigger = np.array(([5, 0] + [0] * 3) * 20)

In [ ]:
stack = []

camera.start_acquisition()
task.write(trigger, auto_start=True)
start = time.time()

while time.time() - start < 5:
    frame = camera.get_frames()
    stack.append(frame)

task.stop()
camera.stop_acquisition()

In [ ]:
stack = np.array([s for st in stack for s in st])
print(stack.shape)

In [ ]:
viewer = napari.view_image(stack)